# ApertureDB

>[ApertureDB](https://docs.aperturedata.io) is a database that stores, indexes, and manages CV related data like images, videos, bounding boxes, embeddings, meta data.

This notebook shows how to use functionality related to the ApertureDB.

To run, you should have a [ApertureDB instance up and running](https://docs.aperturedata.io/HowToGuides/start/Setup).

In [ ]:
%pip install --upgrade --quiet  aperturedb

We shall be using aperturedb Vectorstore to index embeddings from a URL.

We want to use OllamaEmbeddings so we have to import the necessary modules.

Ollama can be set up as a docker container as per the [official guide](https://hub.docker.com/r/ollama/ollama)

In [1]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

#For loading documents from web
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.aperturedata.io")
docs = loader.load()

from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

from langchain_community.vectorstores import ApertureDB
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector_db = ApertureDB.from_documents(documents, embeddings)

llm = Ollama(model="llama2")

In [3]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)


from langchain.chains import create_retrieval_chain

retriever = vector_db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can aperturedata store images?"})
print(response["answer"])

Based on the provided context, ApertureData can store images in various ways. Here are some possibilities:

1. Cloud Storage: ApertureData may use cloud storage services such as Amazon S3, Google Cloud Storage, or Microsoft Azure Blob Storage to store images. This allows for easy scalability and accessibility of the images from anywhere.
2. Database Tables: ApertureData can store images in database tables, possibly using a separate table for each modality (e.g., images, videos, documents). This allows for efficient querying and manipulation of the images based on their metadata and other factors.
3. Object Store: ApertureData may use an object store service such as AWS S3, Google Cloud Storage, or Azure Blob Storage to store images. This allows for easy scalability and accessibility of the images from anywhere.
4. Local Storage: ApertureData can also store images locally on its servers or on customer's premises, depending on their hosting options. This can be useful for customers who r